# 以相似度为训练目标

## 生成训练数据，暂存于本地

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3b

In [0]:
import os
import cv2
import numpy as np
path1 = '05wan3'
path2 = '05wan3b/0'

clocks = dict()
x_train0 = []
y_train0 = []
for filename in os.listdir(path2):
    if 'git' in filename: continue
    #print(path1)
    #ftimestr,number = path1.split('_')
    clocks[int(filename.split('_')[0])]=int(filename.split('_')[1][:-4])
    #clocks[int(ftimestr)]=int(number)
keys = clocks.keys()
for filename in os.listdir(path1):
    if 'git' in filename or 'c.jpg' in filename: continue
    ftime = int(filename.split('_')[0])
    value = max(filter(lambda t:t<ftime,keys))
    x_train0.append(cv2.imread(os.path.join(path1,filename), cv2.IMREAD_GRAYSCALE)[:,:,np.newaxis]/255.0)
    y_train0.append(clocks[value])
    
print(len(x_train0),len(y_train0))

In [0]:
import seaborn as sns
sns.countplot(y_train0)
print(y_train0[:100])

# 转为onehot，分离test和train

In [0]:
from keras.utils.np_utils import to_categorical
y_train0 = to_categorical(y_train0, num_classes = 10)
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train0, y_train0, test_size = 0.1, random_state=2)

In [0]:
print(X_train[0].shape, X_val[0].shape, Y_train[0].shape, Y_val[0].shape)

# 变为5张图片一叠

In [0]:
idxes = []*10
for idx, val in enumerate(y_train0):
  idxes[val].append(idx)

import random

def train_generator():
  for train in range(60000):
    ans = random.randint(0,10)
    data = np.stack(x_train0[random.sample(idxes[ans],5)],axis=2)
    ansy = np.zeros(10)
    ansy[ans] = 1
    yield (data,ansy)

train_dataset = tf.data.Dataset.from_generator(train_generator, (tf.float32, tf.float32), (tf.TensorShape([12,12,5]),tf.TensorShape([10,])))
train_dataset = train_dataset.batch(100)


# 装载Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 1.x
except Exception:
    pass

import tensorflow as tf

# 输入数据生成器

# 构建模型，优化器和损失函数：

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (12,12,5)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 30 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86


from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)
X_train2=np.array(X_train)
X_val2=np.array(X_val)
history = model.fit_generator(datagen.flow(X_train2,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val2,Y_val),
                              verbose = 2, steps_per_epoch=X_train2.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

# 训练并验证模型：